In [1]:
import gmql as gl

In [2]:
# loading of a dataset
data_path = "/home/luca/Documenti/resources/hg_narrowPeaks"
output_path = "/home/luca/Documenti/resources/result_project"

parser = gl.parsers.NarrowPeakParser()

data = gl.GMQLDataset(parser=parser)\
                .load_from_path(path=data_path, meta_load=False)
    
# selects all the data coming from cell K562 and having antibody H3K4me3
condition = (data.MetaField("cell") == 'K562') & (data.MetaField("antibody") == 'H3K4me3')
filtered_data = data.meta_select(predicate=condition) # test

In [3]:
# project on metadata
proj_data = filtered_data.meta_project(attr_list=['antibody', 'cell'])

In [4]:
# project on region fields
new_reg_field = {
    "new_attribute" : (proj_data.RegField("pValue") + proj_data.RegField("signalValue"))/2
}

In [5]:
proj_data = proj_data.reg_project(field_list=['score', 'signalValue', 'pValue'], new_field_dict=new_reg_field)

In [ ]:
# materialization
output_path = "/home/luca/Documenti/resources/result_project"
proj_data = proj_data.materialize(output_path=output_path)

In [ ]:
proj_data.show_info()

In [ ]:
a = new_reg_field['new_attribute']

In [ ]:
e = a.getRegionExtension()

In [ ]:
e.toString()